In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import time

import sys
sys.path.append("../../scripts")

import matplotlib.pyplot as plt
import pandas as pd

from scripts.run_bubblewrap import BubblewrapRun

In [3]:
%load_ext autoreload
import regressions as reg

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
rng = np.random.default_rng()
# np.seterr(all='ignore')

In [5]:
def get_xy1(fit_intercept=False, keep_dead_nodes=False):
    xy_tiles=1
    zscore_x=False
    y_tile_invert=True
    y_true_alignment=True
    x_max_height=False
    with open('../bubblewrap_runs/bubblewrap_run_2023-06-19-11-03-33.pickle', "rb") as fhan:
        br = pickle.load(fhan)
    x_slice_start = 0
    x_slice_end = br.alpha_list.shape[0]
    if y_true_alignment:
        x_slice_start = 1
    if x_max_height:
        x_slice_end = min(x_slice_start + x_max_height, x_slice_end)
    
    node_slice = slice(0,br.alpha_list.shape[1])
    if not keep_dead_nodes:
        node_slice = slice(0,8)
    
    
    X = br.alpha_list[x_slice_start:x_slice_end,node_slice]
    
    if zscore_x:
        X = (X - X.mean(axis=0))/X.std(axis=0)
    
    if fit_intercept:
        X = np.hstack([X, np.ones(shape=[X.shape[0],1])])
    
    y = np.load('../datasets/clock-steadier_farther.npz')['x']
    if y_tile_invert:
        y = np.hstack([y, -y, y]).reshape((-1,1))
    else:
        y = np.hstack([y, y, y]).reshape((-1,1))
    y = y[:X.shape[0]]
    
    X = np.tile(X, reps=(xy_tiles,1))
    y = np.tile(y, reps=(xy_tiles,1))
    return X, y

def get_xy2(fit_intercept=False, keep_dead_nodes=False):
    m = 5000
    n = 5
    
    w = np.array([0, 0, 0, 1, 0]).reshape((-1,1))
    
    X = np.zeros(shape=(m,n))
    y = np.zeros(shape=(m,1))
    for i in range(m):
        X[i,:] = rng.normal(size=5)
    y = X @ w + rng.normal(size=(X.shape[0],1))
    

    if keep_dead_nodes:
        X = np.hstack([X, X])
    
    if fit_intercept:
        X = np.hstack([X, np.ones(shape=[X.shape[0],1])])
    
    return X, y

In [57]:
X, y = get_xy1(fit_intercept=True, keep_dead_nodes=True)

regressors = [
    reg.SymmetricNoisy(d=X.shape[1],forgetting_factor=.9, noise_scale=1e-3),
    reg.WindowFast(d=X.shape[1], window_size=50),
]

predictions = np.zeros([y.shape[0],len(regressors)])
for i in tqdm(range(X.shape[0])):
    for j in range(len(regressors)):
        predictions[i,j] = regressors[j].predict(X[i])
        regressors[j].lazy_observe(X[i], y[i])

100%|███████████████████████████████████| 29998/29998 [00:03<00:00, 8435.32it/s]


In [59]:
%matplotlib qt
fig, ax = plt.subplots()
ax.plot(predictions - y);

In [61]:
%matplotlib qt
fig, ax = plt.subplots()
ax.plot(y)
ax.plot(predictions[:,:ightweight testing])


In [67]:
predictions

array([[       nan,        nan],
       [       nan,        nan],
       [       nan,        nan],
       ...,
       [3.02848309, 3.00000001],
       [3.95871449, 3.99999998],
       [4.97489929, 5.        ]])

## Hyperparameter Tuning

In [26]:
ff_g = [1-.2/x for x in [1,2,3,4,5]]
ns_g = [1, 1e-1, 1e-3, 1e-5, 1e-6, 1e-7]

errors = np.zeros([len(ff_g), len(ns_g)])
for i, ff in enumerate(tqdm(ff_g)):
    for j, ns in enumerate(ns_g):
        predictions = []
        r = reg.SymmetricNoisy(d=X.shape[1],forgetting_factor=ff, noise_scale=ns)
        for k in (range(X.shape[0])):
            predictions.append(r.predict(X[k]))
            r.lazy_observe(X[k], y[k])
        errors[i,j] = np.nanmean((np.array(predictions) - y[:len(predictions),0])**2)


  0%|                                                     | 0/5 [00:00<?, ?it/s]/home/jgould/Documents/Bubblewrap/regressions.py:89: RuntimeWarning: invalid value encountered in matmul
  w = self.D @ self.c
100%|█████████████████████████████████████████████| 5/5 [00:57<00:00, 11.49s/it]


In [32]:
plt.imshow(np.log(errors[:,:4]))